# AMADS Coding Notebooks
## The nPVI measure and an application to Fugue

---

**By (author/s):** Mark Gotham

**For:** Attached to the
[AMADS code library](https://github.com/music-computing/amads/) and
["Keeping Score" book](https://doi.org/10.5281/zenodo.14938027),
but open to all.

**Licence:** MIT.

**Colour key:**
- <font color='green'> Green is for a block of information.
- <font color='purple'> Purple is for an exercise.
- <font color='crimson'> Crimson is for the solution to the exercise above it.

**Learning Objectives:** After completing this notebook, you will be able to ...
- Implement and use the nPVI on 
- Apply this to time values from a musical score or corpus.

If you don't have the required packages, install them now.

As with every libary, you can install in many ways, including:
- `pip` (as in the cell below, uncomment and run one time)
- using the command line (same `pip` command),
- with graphical user interfaces like anaconda-navigator.

For all cases:

In [ ]:
# pip install matplotlib && pip install numpy && pip install seaborn

For the music-specific cases:

In [ ]:
# pip install partitura

Then after install (one time) import every time:

In [ ]:
import matplotlib.pyplot as plt  # If plotting
import numpy as np
from partitura import musicxml_to_notearray  # For score loading
import seaborn as sns  # If plotting in style ;)

## <font color='green'> nPVI

The ‘Normalized Pairwise Variability Index’ (nPVI) measures the variability (broadly, differences) between successive durations in a sequence. Broadly speaking, the nPVI:
1. takes a sequence of durations (sometimes called IOI for ‘inter-onset intervals’)
2. compares each pair of successive items, returning a value of the variability between them based on `(term_1 - term_2) / (term_1 + term_2)`.
3. take the arithmetic average over all those pairs in the sequence.

For more details see Chapter 5: "Time Series".

## <font color='purple'> Exercise: Implement the nPVI

Implement the nPVI and test it out on any sequence of durations.

## <font color='crimson'> Solution

In [ ]:
# Local:

def nPVI(durations: list[float]):
    """A local implementation of the normalized pairwise variability index"""
    numerator = (
        sum(
            [
                abs((k - k1) / ((k + k1) / 2))
                for (k, k1) in zip(durations, durations[1:])
            ]
        )
        * 100
    )
    denominator = len(durations) - 1
    return numerator / denominator

In [ ]:
tresillo = [2, 2, 3]
many_tresillo = tresillo * 100

In [ ]:
nPVI(many_tresillo)

In [ ]:
# On AMADS:

from amads.time.variability import normalized_pairwise_variability_index

normalized_pairwise_variability_index(many_tresillo)

## <font color='purple'> Exercise: Apply to a Score

Apply this logic to the successive durations in a score.

- Use your own choice of score, or
[click here](https://gitlab.com/skalo/mcma/-/raw/master/mcma/bach_js/the_well-tempered_clavier_book_I/BachJS-BWV846-2.mxl?ref_type=heads&inline=false)
to download an example fugue from the MCMA corpus.
- Parse the score and get the note start times. Hint: use `partitura.musicxml_to_notearray` or equivalent.
- Convert start times to durations (difference between successive terms)
- Apply the nPVI

## <font color='crimson'> Solution

In [ ]:
from amads.io.readscore import import_xml

score_path = "../../mcma-master-mcma/mcma/bach_js/the_well-tempered_clavier_book_I/BachJS-BWV846-2.mxl"
# NB: adapt ^^ to your own `score_path`

musicxml_score = import_xml(score_path, show=False)

In [ ]:
def iois(score, remove_zeros: bool = True):
    """From score to list of IOIs."""
    notes = score.get_sorted_notes()  # Notes from all Parts
    diffs = [notes[i].onset - notes[i - 1].onset for i in range(1, len(notes))]
    if remove_zeros:  # filter
        return [item for item in diffs if item > 0]
    return diffs  # unfiltered

deltas = iois(musicxml_score)
nPVI(deltas)

## <font color='purple'> Exercise: Expand to a Corpus, Refine to a Section

If we can do this once, we can do it across a corpus.
That will help give some meaningful context for individual observations.

And if we can do this for a whole piece, we can do it for a small part thereof.
That will help make our analysis more targetted.

For this task:
- Get all fugues from WTC book 1 (via [the MCMA corpus](https://gitlab.com/skalo/mcma))
- Calculate nPVI for the opening subject and answer phases (the starting phase with 1 voice only, then what follows with 2 voices)
- Apply nPVI to the combination of note entries during those phases.

In [ ]:
# Here's how you might start:
def get_MCMA_subject_answer_nPVIs(your_base_path: str, plot: bool = True):
    """
    Get all fugues from WTC book 1 and calculate nPVI for the subject and answer phase.
    The arg `your_base_path mcma_base` should 
    lead up to and excluding `mcma-master-mcma`,
    e.g., "/Users/<Your name>/<Documents?>/"
    """
    pass

## <font color='crimson'> Solution

In [ ]:
from amads.core.basics import Part, Note

def where_each_part_starts(
    parsed_score = musicxml_score,
    part_numbers: bool = False
):
    """
    Retrieve the timepoint of each part's first note.
    Return that list of timepoints or
    (if `part_numbers == True`)
    a tuple with part number and timepoint.
    """
    if part_numbers:
        return [
            (index, part.list_all(Note)[0].onset) for index, part in enumerate(parsed_score.find_all(Part))
        ]
    else:
        return [part.list_all(Note)[0].onset for part in parsed_score.find_all(Part)]

In [ ]:
where_each_part_starts(part_numbers=False)

In [ ]:
where_each_part_starts(part_numbers=True)

In [ ]:
def get_MCMA_subject_answer_nPVIs(your_base_path: str, plot: bool = True):
    """
    Get all fugues from WTC book 1 and calculate nPVI for the subject and answer phase.
    The arg `your_base_path mcma_base` should 
    lead up to and excluding `mcma-master-mcma`,
    e.g., "/Users/<Your name>/<Documents?>/"

    This implementation prioritises clarity over efficiency,
    e.g., with part entrances calculated first, separately.
    """
    mcma_directory = your_base_path + "mcma-master-mcma/mcma/bach_js/the_well-tempered_clavier_book_I/"
    bwvs = range(846, 869 + 1)
    subject_answer_label = []  # We'll store a tuple for each case in here
    for bwv in bwvs:

        file_name = f"BachJS-BWV{bwv}-2.mxl"
        score_path = mcma_directory + file_name

        # Part entrances separately first.
        fugue_score = import_xml(score_path)
        part_entrances = where_each_part_starts(fugue_score, part_numbers=False)
        part_entrances.sort()
    
        try:
            subject_entrance, answer_entrance, end = part_entrances[0:3]  # Part entrances 1, 2, and 3 (we end there)
        except:  # Fails if there are fewer than 3 voice. There is one case of a 2 voice fugue in there, BWV855.
            assert len(part_entrances) == 2  # Double check :)
            print(f"Skipping the 2-voice fugue BWV{bwv}")
            continue

        # Now the combined version for all onsets
        note_onsets = [n.onset for n in fugue_score.find_all(Note)]
        
        subject = list(set(x for x in note_onsets if (subject_entrance <= x <= answer_entrance)))
        subject.sort()
        subject_durations = [b - a for a, b in zip(subject[:-1], subject[1:])]
        subject_nPVI = nPVI(subject_durations)

        answer = list(set(x for x in note_onsets if (answer_entrance <= x <= end)))
        answer.sort()
        answer_durations = [b - a for a, b in zip(answer[:-1], answer[1:])]
        answer_nPVI = nPVI(answer_durations)

        # Only label for lines that go up.
        label = ""
        if answer_nPVI > subject_nPVI:
            label = f"BWV{bwv}"

        subject_answer_label.append(
            (subject_nPVI, answer_nPVI, label)
        )

    if plot:
        plot_tuple_changes(subject_answer_label)
    else:
        subject_mean = np.mean([x[0] for x in subject_answer_label])
        subject_std = np.std([x[0] for x in subject_answer_label])

        answer_mean = np.mean([x[1] for x in subject_answer_label])
        answer_std = np.std([x[1] for x in subject_answer_label])

        return subject_mean, subject_std, answer_mean, answer_std

In [ ]:
import seaborn as sns

def plot_tuple_changes(data):
    """
    Plots changes in y-coordinates from a list of tuples.

    Args:
        data: A list of tuples, where each tuple contains two y-coordinate values.
    
    Each tuple represents a change from one state to the next with in the same piece.
    """
    # Set a colorblind-friendly palette (e.g., 'colorblind')
    palette = sns.color_palette("colorblind", n_colors=24)

    plt.figure(figsize=(8, 6))  # Adjust figure size if needed

    for i in range(len(data)):
        fugue = data[i]
        try:
            colour = palette[i]
        except:
            print(f"fail on colour index {i}")
            colour = palette[0]
        plt.plot(
            [1, 2],  # "x" values
            [fugue[0], fugue[1]],  # ys
            marker='o', linestyle='-', label=fugue[2], color=colour # Design
        )

    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Going up:")
    plt.xticks([1, 2], ["subject", "answer"])
    plt.ylabel("nPVI")
    plt.title("nPVI Change from Subject to Answer")
    plt.grid(True)
    plt.tight_layout()

    plt.savefig("./npvi_subject_answer.pdf")

In [ ]:
get_MCMA_subject_answer_nPVIs(your_base_path="../../", plot=True)  # Put your path here!